In [ ]:
from __future__ import annotations
from typing import *
if TYPE_CHECKING:
    pass
from numpy.lib.function_base import iterable
from src.Kernel import GaussKernel
from src.EllipsoidRenderer import EllipsoidRenderer
from src.GridBuilder import ImageGridBuilder
from read_data import load_position_data
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
data = np.load('data/toastgitter/all_data.npy') # Shape [STEPS, NUM_REL]
p_pos, c_pos = load_position_data('data/toastgitter/LOGFILE_unified_coordinates.txt')
print(data.shape)


In [ ]:
NUM_SAMPLES, NUM_RELATIONS = data.shape
SPEED_OF_SOUND = 1_484_000.0 # [mm/s]
SAMPLE_RATE = 10_000_000.0 # [1/s]
ECHO_START_TIME = 0.0 # [s]

In [ ]:
plt.plot(np.mean(data, axis=1))

In [ ]:
plt.plot(np.std(data, axis=1))